# Black–Litterman Bayesian Portfolio Optimizer

This notebook walks through data fetching, historical mean–variance (MV) estimation, constructing Black–Litterman (BL) posterior from views, and comparing efficient frontiers and optimal portfolios.

Sections:
1. Setup and parameters
2. Data loading and preprocessing
3. Historical MV estimates (μ, Σ)
4. Market-implied equilibrium returns (π)
5. Views and BL posterior
6. Efficient frontiers (MV vs BL)
7. Optimal portfolios and weights comparison

## 1. Setup and parameters

We’ll set analysis parameters (tickers, dates, risk-free rate, BL hyperparameters) and import project modules.

In [1]:
# Parameters and imports
RISK_FREE = 0.0
TAU = 0.05
DELTA = 2.5
TICKERS = ["AAPL", "MSFT", "GOOGL", "AMZN"]
START = "2018-01-01"

import pandas as pd
from src import data_loader as dl
from src import implied_returns as ir
from src import views as vw
from src import optimizer as opt
from src import plots
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'src'

In [ ]:
# 2) Data loading
prices = dl.fetch_prices(TICKERS, start=START).prices
prices.tail()

In [ ]:
# 3) Historical MV estimates
mu_hist, Sigma_hist = dl.returns_and_covariance(prices, method='log')
print('mu_hist:')
display(mu_hist)
print('Sigma_hist shape:', Sigma_hist.shape)

In [ ]:
# 4) Market-implied equilibrium returns (π)
market_caps = {t: 1.0 for t in TICKERS}  # placeholder; replace with real caps for accuracy
w_mkt = ir.market_caps_to_weights(market_caps)
pi = ir.compute_pi(Sigma_hist, w_mkt, DELTA)
print('pi (equilibrium returns):')
display(pi)

In [ ]:
# 7) Optimal (max Sharpe) portfolios under MV and BL
w_mv, r_mv, s_mv, sh_mv = opt.max_sharpe_portfolio(mu_hist, Sigma_hist)
w_bl, r_bl, s_bl, sh_bl = opt.max_sharpe_portfolio(bl.mu_bl, bl.Sigma_bl)
print('MV max Sharpe metrics:', {'ret': r_mv, 'risk': s_mv, 'sharpe': sh_mv})
print('MV weights:')
display(w_mv.sort_values(ascending=False))
print('BL max Sharpe metrics:', {'ret': r_bl, 'risk': s_bl, 'sharpe': sh_bl})
print('BL weights:')
display(w_bl.sort_values(ascending=False))

In [ ]:
# 6) Efficient frontiers and plot
curves = opt.compare_mv_vs_bl(mu_hist, Sigma_hist, bl.mu_bl, bl.Sigma_bl)
fig, ax = plots.plot_frontiers(*curves.values())
plt.show()

# Simple comparison table at equal grid index (midpoint)
mv_df, _ = curves['mv']
bl_df, _ = curves['bl']
mid = len(mv_df)//2
comp = pd.DataFrame({
    'MV_return': [mv_df.loc[mid, 'return']],
    'MV_risk': [mv_df.loc[mid, 'risk']],
    'BL_return': [bl_df.loc[mid, 'return']],
    'BL_risk': [bl_df.loc[mid, 'risk']],
})
comp

In [ ]:
# 5) Views and BL posterior
# Example: AAPL expected to outperform MSFT by 2% annually
P_row, q = vw.relative_view(TICKERS, long='AAPL', short='MSFT', magnitude=0.02)
P, Q = vw.build_PQ([(P_row, q)])
bl = opt.black_litterman_posterior(Sigma_hist, pi, P, Q, tau=TAU)
print('BL posterior mu:')
display(bl.mu_bl)

In [ ]:
# 7b) Compare weights visually
ax = pd.DataFrame({'MV': w_mv, 'BL': w_bl}).plot(kind='bar', figsize=(8,4), title='Max Sharpe Weights: MV vs BL')
plt.tight_layout()
plt.show()